In [40]:
import requests
import pandas as pd

def get_1day_sticks_of_1year(coin, year):
    url = f"https://api.upbit.com/v1/candles/days?market=KRW-{coin}&to={year}-12-31%2012%3A59%3A59&count=200"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    print(response)
    df = pd.DataFrame(response.json()).sort_values('candle_date_time_utc').reset_index().drop(columns='index')

    df2 = pd.DataFrame()
    url = f"https://api.upbit.com/v1/candles/days?market=KRW-{coin}&to={year}-06-14%2012%3A59%3A59&count=200"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    print(response)
    df2 = pd.DataFrame(response.json()).sort_values('candle_date_time_utc').reset_index().drop(columns='index')   
    
    result = pd.concat([df2,df]).reset_index().drop(columns='index').drop_duplicates()
    return result


In [41]:
def get_profit_rate(df):
    
    # DMI, ADX 지표
    import pandas as pd
    import ta
    import ta.utils
    high = df['high_price'].astype('float')
    low = df['low_price'].astype('float')
    close = df['trade_price'].astype('float')
    adx = ta.trend.ADXIndicator(high=high, low=low, close=close)

    df['ADX'] = adx.adx().astype('float')
    df['PDI'] = adx.adx_pos().astype('float')
    df['MDI'] = adx.adx_neg().astype('float')
    df['candle_date_time_utc'] = pd.to_datetime(df['candle_date_time_utc'])

    df['SMA_10'] = df['ADX'].rolling(window=10).mean()
    df['ADX_SMA_width'] = df['ADX'] - df['SMA_10']
    df.loc[35, 'ADX_SMA_width'] = df.loc[36, 'ADX_SMA_width']
    df['ADX_SMA_slope'] = (df['ADX_SMA_width'] - df['ADX_SMA_width'].shift().fillna(df['ADX_SMA_width'])) /2
    df.loc[35, 'ADX_SMA_slope'] = df.loc[36, 'ADX_SMA_slope']


    # 볼린저 지표
    import matplotlib.pyplot as plt
    import ta.volatility as tav

    df['upper'] = tav.bollinger_hband(df['trade_price'])
    df['lower'] = tav.bollinger_lband(df['trade_price'])
    df['middle'] = tav.bollinger_mavg(df['trade_price'])
    df['bol_width'] = tav.bollinger_wband(df['trade_price'])
    df = df[27:].reset_index(drop=['index'])

    df['bol_width_slope'] = (df['bol_width'] - df['bol_width'].shift().fillna(df['bol_width'])) /2
    df['bol_width_slope'] = df['bol_width_slope']
    

    #매수타이밍
    df['ADX_sign'] = False
    for i in range(len(df)):
        if df.loc[i, 'ADX_SMA_slope']>=0:
            df.loc[i ,'ADX_sign'] = True
        else:
            continue
    df['DMI_sign'] = False
    df.loc[df['PDI'] - df['MDI']>0, 'DMI_sign'] = True

    df['bol_shrink'] = False
    df['break_point'] = False
    df.loc[df['bol_width_slope']<0, 'bol_shrink'] = True
    df.loc[df['trade_price']>=df['upper'], 'break_point'] = True

    breakpoint_found = False
    cnt = 0
    for i in range(len(df)):
        if df.loc[i,'break_point'] :
            if not breakpoint_found:
                breakpoint_found = True
                cnt+=1
            else:
                df.loc[i,'break_point'] = False
                continue
        else: 
            if df.loc[i,'bol_shrink']:
                breakpoint_found = False
            else:
                continue
    df.loc[df['break_point'] & df['ADX_sign'] & df['DMI_sign'], 'stat']= 'buy'

    
    ### 매도타이밍
    df['comparison_ADX'] = df.apply(lambda row: '+' if row['ADX'] - row['SMA_10'] >= 0 else '-', axis=1)
    df.loc[(df['comparison_ADX'].shift(1) == '+') & (df['comparison_ADX'] == '-'), 'stat'] = 'sell'
    

    ### 트레이딩 결과
    df_trade_info = pd.concat([df.loc[df['stat']=='buy'],df.loc[df['stat']=='sell']]).sort_index().reset_index().drop(columns='index')
    trade_fee = 0.0005
    stat = 'buy'
    trade_lst = list()
    for i in range(len(df_trade_info)):
        
        if stat == 'buy':
            if df_trade_info.loc[i ,'stat'] == 'buy':
                stat='sell'
                trade_dict = dict()
                trade_dict['buy_date'] = df_trade_info.loc[i ,'candle_date_time_utc']
                trade_dict['buy'] = df_trade_info.loc[i ,'trade_price'] * (1+trade_fee)

                continue

            else:
                df_trade_info.drop(index=i, inplace=True)

        else:
            if df_trade_info.loc[i ,'stat'] == 'sell':
                stat='buy'
                trade_dict['sell_date'] = df_trade_info.loc[i ,'candle_date_time_utc']
                trade_dict['sell'] = df_trade_info.loc[i ,'trade_price'] * (1-trade_fee)

                trade_lst.append(trade_dict)
                continue
            else:
                df_trade_info.drop(index=i, inplace=True)

    df_trade_result = pd.DataFrame(trade_lst)
    df_trade_result['profit'] = df_trade_result['sell']-df_trade_result['buy']
    df_trade_result['profit_rate'] = df_trade_result['profit']/df_trade_result['buy']
    df_trade_result

    return df_trade_result


In [42]:
import pandas as pd
# 코인 리스트 가져오기
import requests

url = "https://api.upbit.com/v1/market/all?isDetails=true"

headers = {"accept": "application/json"}

res = requests.get(url, headers=headers)

df = pd.DataFrame(res.json())
df = df.loc[df['market'].str.contains('KRW')]
df['market'] = df['market'].str.strip('KRW-')
coinlist = list(df['market'])
coinlist

['BTC',
 'ETH',
 'NEO',
 'MTL',
 'XRP',
 'ETC',
 'SNT',
 'AVES',
 'XEM',
 'QTUM',
 'LS',
 'STEEM',
 'XLM',
 'ARD',
 'A',
 'STORJ',
 'GRS',
 'ADA',
 'SBD',
 'PO',
 'BTG',
 'ICX',
 'EOS',
 'TRX',
 'SC',
 'ONT',
 'ZIL',
 'POLYX',
 'ZRX',
 'LOOM',
 'BCH',
 'BAT',
 'IOST',
 'CVC',
 'IQ',
 'IOTA',
 'HIFI',
 'ONG',
 'GAS',
 'UPP',
 'ELF',
 'NC',
 'BSV',
 'THETA',
 'QKC',
 'BTT',
 'MOC',
 'TFUEL',
 'MANA',
 'AN',
 'AERGO',
 'ATOM',
 'TT',
 'GAME2',
 'MBL',
 'AXP',
 'HBA',
 'MED',
 'ML',
 'STPT',
 'ORBS',
 'VET',
 'CHZ',
 'STMX',
 'DKA',
 'HIVE',
 'AVA',
 'AHT',
 'LIN',
 'XTZ',
 'BORA',
 'JST',
 'CRO',
 'TON',
 'SXP',
 'HUNT',
 'DOT',
 'MVL',
 'STRAX',
 'AQT',
 'GLM',
 'META',
 'FCT2',
 'CB',
 'SAND',
 'HPO',
 'DOGE',
 'STRIKE',
 'PUNDIX',
 'FLO',
 'AXS',
 'STX',
 'XEC',
 'SOL',
 'MATIC',
 'AAVE',
 '1INCH',
 'ALGO',
 'NEA',
 'AVAX',
 'T',
 'CELO',
 'GMT',
 'APT',
 'SHIB',
 'MAS',
 'ARB',
 'EGLD',
 'SUI',
 'GRT',
 'BLU',
 'IMX',
 'SEI',
 'MINA',
 'CTC',
 'AST',
 'ID',
 'PYTH',
 'MNT',
 'AKT',
 '

In [43]:
# year = 2022
# df_total_trade = pd.DataFrame() 
# fail_list = list()   
# for coin in coinlist:
#     try:
#         df = get_1day_sticks_of_1year(coin, year)
#         df_trade = get_profit_rate(df)
#         # df_trade['coin'] = coin
#         df_trade.insert(0, 'coin', coin)
#         df_total_trade = pd.concat([df_total_trade, df_trade])
#         print(coin, '매매 정보 입력완료')
#     except:
#         fail_list.append(coin)
#         continue
# df_total_trade

In [44]:
# df_total_trade.to_csv(f'DATA/{year}_total_trade_DATA.csv')

In [45]:
year = 2022
df_total_trade =pd.read_csv(f'DATA/{year}_total_trade_DATA.csv')

In [46]:
# 수익률 퍼센트
temp = df_total_trade.copy()
temp['profit_rate_plus'] = temp['profit_rate'] + 1
res = (temp.groupby('coin')['profit_rate_plus'].prod()-1)*100
res

coin
1INCH   -11.226395
AAVE    -70.740023
ADA     -24.906003
AERGO   -42.694510
AHT     -27.624217
           ...    
XLM     -24.093847
XRP       9.303943
XTZ     -44.356003
ZIL      85.398707
ZRX     -82.127172
Name: profit_rate_plus, Length: 88, dtype: float64

In [47]:
# 각 종목에 1원 투자했을 때 결과 1 * 88
(1 + 1*(res/100)).sum()

64.35084229918806

In [51]:
# 각 투자결과에 대한 describe
(1 + 1*(res/100)).describe()

count    88.000000
mean      0.731260
std       0.367766
min       0.178728
25%       0.513867
50%       0.692748
75%       0.870632
max       2.292842
Name: profit_rate_plus, dtype: float64